In [7]:
import pandas as pd
import os
from pathlib import Path

In [8]:
base_dir = Path('..') / 'evaluation'  # Adjust path if needed
tasks = ['task_1', 'task_2', 'task_3']
modality_list = ['xml', 'json', 'image']

accuracy_records = []
invalid_records = []

In [9]:
for task in tasks:
    task_path = base_dir / task
    for pe_type in task_path.iterdir():
        if not pe_type.is_dir():
            continue
        for model in pe_type.iterdir():
            if not model.is_dir():
                continue
            for file in model.glob("*.csv"):
                modality = file.stem.lower()  # e.g., xml, json, image
                df = pd.read_csv(file)
                
                # Accuracy
                acc = round(df['score'].mean(), 2)
                accuracy_records.append({
                    'model': model.name,
                    'pe_type': pe_type.name,
                    'task': task,
                    'modality': modality,
                    'accuracy': acc
                })

                # Count invalids
                invalid_count = int((df['answer'].astype(str).str.lower() == "invalid").sum())
                invalid_records.append({
                    'model': model.name,
                    'pe_type': pe_type.name,
                    'task': task,
                    'modality': modality,
                    'invalid_count': invalid_count
                })

In [10]:
# Convert to DataFrames
acc_df = pd.DataFrame(accuracy_records)
inv_df = pd.DataFrame(invalid_records)

# Pivot for final tables
acc_pivot = acc_df.pivot_table(index=['model', 'pe_type'], 
                                columns=['task', 'modality'], 
                                values='accuracy')

inv_pivot = inv_df.pivot_table(index=['model', 'pe_type'], 
                                columns=['task', 'modality'], 
                                values='invalid_count', fill_value=0)

# Optional: Flatten MultiIndex columns
acc_pivot.columns = [f'{t.upper()} {m.upper()}' for t, m in acc_pivot.columns]
inv_pivot.columns = [f'{t.upper()} {m.upper()}' for t, m in inv_pivot.columns]

# Format invalid table values as integers
inv_pivot = inv_pivot.astype(int)

# Display
print("=== Accuracy Table ===")
display(acc_pivot.reset_index())

print("\n=== Invalid Answer Count Table ===")
display(inv_pivot.reset_index())

=== Accuracy Table ===


,model,pe_type,TASK_1 IMAGE,TASK_1 JSON,TASK_1 XML,TASK_2 IMAGE,TASK_2 JSON,TASK_2 XML,TASK_3 IMAGE,TASK_3 JSON,TASK_3 XML
0,gemma3-4b,null-shot,0.54,0.46,0.51,0.40,0.30,0.31,0.20,0.18,0.32
1,gemma3-4b,null-shot-cot,0.53,0.57,0.55,0.38,0.31,0.32,0.16,0.17,0.33
2,gemma3-4b,zero-shot,0.55,0.56,0.57,0.40,0.35,0.36,0.18,0.14,0.31
3,gemma3-4b,zero-shot-cot,0.51,0.51,0.55,0.39,0.30,0.40,0.21,0.19,0.32
4,gpt_4-1-mini,null-shot,0.55,0.52,0.53,0.60,0.46,0.42,0.44,0.70,0.76
5,gpt_4-1-mini,null-shot-cot,0.55,0.56,0.54,0.64,0.47,0.43,0.42,0.71,0.78
6,gpt_4-1-mini,zero-shot,0.58,0.51,0.50,0.59,0.41,0.45,0.50,0.73,0.83
7,gpt_4-1-mini,zero-shot-cot,0.59,0.55,0.55,0.60,0.51,0.47,0.47,0.68,0.81
8,gpt_4o,null-shot,0.65,0.50,0.50,0.71,0.50,0.47,0.79,0.62,0.64
9,gpt_4o,null-shot-cot,0.65,0.48,0.52,0.71,0.49,0.49,0.76,0.64,0.65



=== Invalid Answer Count Table ===


,model,pe_type,TASK_1 IMAGE,TASK_1 JSON,TASK_1 XML,TASK_2 IMAGE,TASK_2 JSON,TASK_2 XML,TASK_3 IMAGE,TASK_3 JSON,TASK_3 XML
0,gemma3-4b,null-shot,1,0,3,7,42,3,20,92,3
1,gemma3-4b,null-shot-cot,0,0,3,14,47,5,42,92,0
2,gemma3-4b,zero-shot,0,1,2,1,43,4,27,99,1
3,gemma3-4b,zero-shot-cot,0,1,9,9,43,5,74,91,1
4,gpt_4-1-mini,null-shot,0,0,0,0,0,0,0,0,0
5,gpt_4-1-mini,null-shot-cot,0,0,0,0,0,0,0,0,0
6,gpt_4-1-mini,zero-shot,0,0,0,0,0,0,0,0,0
7,gpt_4-1-mini,zero-shot-cot,0,0,0,0,0,0,0,0,0
8,gpt_4o,null-shot,0,0,0,0,0,0,0,0,0
9,gpt_4o,null-shot-cot,0,0,0,0,0,0,0,0,0


In [12]:
inv_summary = inv_df.groupby(['model', 'task', 'modality'])['invalid_count'].sum().reset_index()
inv_summary_pivot = inv_summary.pivot(index='model', columns=['task', 'modality'], values='invalid_count').fillna(0).astype(int)
inv_summary_pivot.columns = [f"{task.upper()} {modality.upper()}" for task, modality in inv_summary_pivot.columns]
inv_summary_pivot = inv_summary_pivot.reset_index()
print("\n=== Invalid Answer Count Table (No PE Type) ===")
display(inv_summary_pivot)



=== Invalid Answer Count Table (No PE Type) ===


,model,TASK_1 IMAGE,TASK_1 JSON,TASK_1 XML,TASK_2 IMAGE,TASK_2 JSON,TASK_2 XML,TASK_3 IMAGE,TASK_3 JSON,TASK_3 XML
0,gemma3-4b,1,2,17,31,175,17,163,374,5
1,gpt_4-1-mini,0,0,0,0,0,0,0,0,0
2,gpt_4o,0,0,0,0,0,0,0,0,0
3,llava-7b,0,0,1,16,126,5,116,265,138
4,qwen2.5-vl-3b,4,2,12,67,101,36,92,198,134
5,qwen2.5-vl-7b,0,1,21,9,145,7,139,479,5


In [ ]:
# # Save both tables into one Excel file with separate sheets
# output_path = Path('all_task_results.xlsx')  # Save in current directory

# with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
#     acc_pivot.reset_index().to_excel(writer, index=False, sheet_name='Accuracy')
#     inv_pivot.reset_index().to_excel(writer, index=False, sheet_name='Invalid Counts')
#     inv_summary_pivot.to_excel(writer, index=False, sheet_name='Invalid Counts (No PE Type)')
# print(f"Saved results to {output_path}")


Saved results to all_task_results.xlsx


In [1]:
task1_acc_df = acc_df[acc_df['task'] == 'task_1']


NameError: name 'acc_df' is not defined